<a href="https://colab.research.google.com/github/evaie/evaie/blob/main/process_inscriptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get authentication for reading the google sheet

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Read the google sheet and transform it in a dataframe properly


In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_inscriptions = pd.DataFrame.from_records(rows,columns=rows[0])
df_inscriptions = df_inscriptions.drop(0).reindex()

detect the malformed naissance columsns and change the date formate

In [ ]:
df_inscriptions.dropna()
df_inscriptions['naissance'].count()
df_ddl_1 = df_inscriptions[df_inscriptions['naissance']=='0000-00-00']
df_ddl_1

,saison,statut,date_statut,civil,nom,prenom,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total
47,2021-2022,Validé,2021-09-22 15:55:35,Monsieur,EXBRAYAT,JEAN-LOUIS,jexbrayat@icloud.com,06 69 05 22 63?,71 rue Victor Hugo,92400,Courbevoie,FR,FR,0000-00-00,retrairé,,,,,,,,,,,,Madame,EXBRAYAT,ELYANE,elexbrayat@gmail.com,06 64 26 24 16?,Fanny Exbrayat,2020-09-04,non,on,on,on,on,on,renouvellement,432020,Séniors + Loisirs + CE,non,non,460.00,0.00,,460.00,0.00,0.00,460.00,1,0.00,0.00,0.00,,0.00,0.00,,0.00,460.00
69,2021-2022,Validé,2021-09-25 09:20:07,Monsieur,GUG,PASCAL,pascalgugperso@orange.fr,607481660,60 avenue GALLIENI,92400,COURBEVOIE,FR,FR,0000-00-00,Avocat,Retraité,,,,,,,,,,,Madame,GUG,ANNIE,anniechentrier.perso@orange.fr,643083818,,0000-00-00,non,on,on,on,on,on,renouvellement,,Séniors + Loisirs + CE,non,non,460.00,0.00,,460.00,0.00,0.00,460.00,1,0.00,0.00,0.00,,0.00,0.00,,0.00,460.00
85,2021-2022,Validé,2021-09-22 16:06:25,Monsieur,KALDY,PIERRE,pkaldy@laposte.net0,783192237,74 rue Duhesme,75018,Paris,FR,FR,0000-00-00,journaliste scientifique,,,,,,,,,,,,Madame,KALDY,FRANÇOISE,francoisem13@hotmail.com,607910694,Anne-Sophie Laugier,2019-09-04,non,on,on,on,on,on,renouvellement,141819,Séniors + Loisirs + CE,non,non,480.00,0.00,,480.00,0.00,0.00,480.00,1,0.00,0.00,0.00,,0.00,0.00,,0.00,480.00
101,2021-2022,Validé,2021-09-24 17:47:29,Monsieur,MEREL,ROMÉO,celine.morel864@orange.fr,787326266,"4, rue Héreold",92250,La Garenne Colombes,FR,FR,0000-00-00,classe de 3ème,La Trinité Neuilly,Madame,MEREL,CÉLINE,celine.morel864@orange.fr,626553638,Monsieur,MEREL,PHILIPPE,philippe.merel4@wanadoo.fr,612992415,Madame,MEREL,CÉLINE,celine.morel864@orange.fr,626553638,Pillard,0000-00-00,non,on,on,on,on,on,renouvellement,525123,J11 / J16 - né(e) en 2005 et après,non,non,330.00,60.00,pass 92,270.00,0.00,0.00,0.00,0,0.00,0.00,0.00,,0.00,0.00,,0.00,0.00
136,2021-2022,Validé,2021-09-22 16:37:08,Madame,TOKAR,OLENA,l_tokar@yahoo.com,622773052,17 bd Georges Seurat,92200,Neuilly-sur-Seine,FR,FR,0000-00-00,Consultante,,,,,,,,,,,,Monsieur,HARBULA,PÉTER,pharbula@hotmail.com,616215594,Bordon,0000-00-00,non,on,on,on,on,on,renouvellement,549080,Séniors + Loisirs + CE,non,oui,480.00,220.00,Rando annulé (Descente de Cher),260.00,0.00,0.00,0.00,0,260.00,0.00,0.00,,0.00,0.00,,0.00,260.00


In [ ]:
df_ddl_2 =df_inscriptions[df_inscriptions['naissance']=='']
df_ddl_2

,saison,statut,date_statut,civil,nom,prenom,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total


In [ ]:
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='0000-00-00']
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='']

In [ ]:
from datetime import datetime
df_inscriptions['naissance']= df_inscriptions[df_inscriptions['naissance']!='0000-00-00']['naissance'].\
apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [ ]:
df_inscriptions['sexe'] = df_inscriptions['civil'].apply(lambda x : 'H' if x == 'Monsieur' else 'F')

In [ ]:
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='^parent')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='^contact')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='[a-z]*cheque[a-z]*')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(['documentation', 'benevolat', 'nager', 'assurance',\
                                                                'caution_nom', 'caution', 'especes', 'autre_details',\
                                                                'montant_total','saison','statut','date_statut','profession',\
                                                                'entreprise','reglement','garage','montant','reduction',\
                                                                'motifreduction','netapayer', 'paiement_helloasso','virement',\
                                                                'couponsport','pass92','autre'])]

In [ ]:
from datetime import date,timedelta
yesterday = date.today() - timedelta(days=1)
yester_str = yesterday.strftime('%Y-%m-%d')

In [ ]:
df_inscriptions['docteur_date'] = df_inscriptions['docteur_date'].apply(lambda x : yester_str if x =='0000-00-00' else x)

In [ ]:
from datetime import datetime
df_inscriptions['docteur_date']= df_inscriptions['docteur_date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [ ]:
def reprocess_add(input_str):
  input_str = input_str.str.replace('?','  ')
  #input_str = input_str.replace('\n',' ')

In [ ]:
df_inscriptions['docteur_nom'] = df_inscriptions['docteur_nom'].str.upper()
df_inscriptions['ville'] = df_inscriptions['ville'].str.upper()
df_inscriptions['adresse'] = df_inscriptions['adresse'].str.upper()
#df_inscriptions['adresse'] = df_inscriptions['adresse'].apply(lambda x : reprocess_add(x))
#df_inscriptions['adresse'] = df_inscriptions['adresse'].apply(lambda x : reprocess_add(x))

Split the address

In [ ]:
df_inscriptions['NumeroVoie'] = df_inscriptions['adresse'].str.split(n=2,expand=True)[0]
df_inscriptions['TypeVoie'] = df_inscriptions['adresse'].str.split(n=2,expand=True)[1]
df_inscriptions['LibelleVoie'] = df_inscriptions['adresse'].str.split(n=2,expand=True)[2]


Read the tab with the licences already taken

In [ ]:
df_inscriptions.head()

,civil,nom,prenom,email,tel,adresse,cp,ville,pays,nationalite,naissance,docteur_nom,docteur_date,handicap,type,licence,categorie,optioncomp,sexe,NumeroVoie,TypeVoie,LibelleVoie
1,Monsieur,ACHACHE,LAURENT,laurentachache@gmail.com,620806479,147 RUE DE JAVEL,75015,PARIS,FR,FR,23/08/1958,FRANCOISE BESSE,21/09/2020,non,renouvellement,38647,Séniors + Loisirs + CE,non,H,147,RUE,DE JAVEL
2,Monsieur,ALEXANDRE,OLIVIER,olivier.alexandre2@wanadoo.fr,676450830,115 RUE VICTOR HUGO,92270,BOIS-COLOMBES,FR,FR,28/05/1958,GAUZERAN,18/06/2021,non,renouvellement,44215,Séniors + Loisirs + CE,non,H,115,RUE,VICTOR HUGO
3,Monsieur,ALVIN,MAXIMILIEN,m.alvin@orange.fr,648752437,17 AVENUE DE CHANTILLY,60300,SENLIS,FR,FR,09/02/1984,CHRISTOPHE BEAUBE,09/09/2021,non,renouvellement,244705,Séniors + Loisirs + CE,non,H,17,AVENUE,DE CHANTILLY
4,Madame,ANDRE,DAPHNÉ,daphne.andre09@gmail.com,788590780,103 RUE RAYMOND RIDEL,92400,COURBEVOIE,FR,FR,28/06/2009,MANTEAU,08/10/2019,non,renouvellement,539073,J11 / J16 - né(e) en 2005 et après,non,F,103,RUE,RAYMOND RIDEL
5,Madame,ANDREAN,CHARLOTTE,charlotte.portmann@gmail.com,629462075,133 AVENUE DE LA RÉPUBLIQUE,92400,COURBEVOIE,FR,FR,13/03/1974,ANNE PETIT,28/05/2021,non,adhésion,,Séniors + Loisirs + CE,non,F,133,AVENUE,DE LA RÉPUBLIQUE


In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').worksheet('licences_deja_prises')

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_licences = pd.DataFrame.from_records(rows,columns=rows[0])

In [ ]:
df_licences = df_licences.drop(0).reindex()

In [ ]:
import unicodedata

def remove_accents(input_str):
  # remove simple quote and '-
  input_str = input_str.replace('\'','')
  input_str = input_str.replace('-','')
  nfkd_form = unicodedata.normalize('NFKD', input_str)
  return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

remove the accent for 1st name and last name


In [ ]:
df_licences['NOM'] = df_licences['NOM'].apply(lambda x : remove_accents(x))
df_licences['PRENOM'] = df_licences['PRENOM'].apply(lambda x : remove_accents(x))
df_inscriptions['nom'] = df_inscriptions['nom'].apply(lambda x : remove_accents(x))
df_inscriptions['prenom'] = df_inscriptions['prenom'].apply(lambda x : remove_accents(x))



In [ ]:
df_licencies = pd.DataFrame(data=df_licences['NOM'] + '_' + df_licences['PRENOM'] + '_' + df_licences['DATE NAISSANCE'])

An then build a list of licenced persons

In [ ]:
licencies = df_licencies.to_dict('list')[0]

Check who already has a licence


In [ ]:
df_inscriptions['key'] = df_inscriptions['nom'] + '_' + df_inscriptions['prenom'] + '_' + df_inscriptions['naissance']
df_inscriptions['licence_prise'] = df_inscriptions['key'].apply(lambda x : True if x in licencies else False)

Then save the result : only the licence not already taken

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').worksheet("Fitogram")

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_fito = pd.DataFrame.from_records(rows,columns=rows[8])
df_fito = df_fito[df_fito['Nom']!=''].drop(8).drop(9)
df_fito = df_fito[df_fito['Email']!='']

In [ ]:
l_fito = pd.DataFrame(df_fito['Email']).to_dict('list')['Email']

Then give a status to the Fitogram account by comparing the address mails


In [ ]:
df_inscriptions['is_in_fitogram'] = df_inscriptions['email'].apply(lambda x : True if x in l_fito else False)

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('re-processed')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="re-processed", rows="350", cols="20")

from gspread_dataframe import set_with_dataframe
#set_with_dataframe(inscription_reprocessed, df_inscriptions[df_inscriptions['licence_prise']==False])
set_with_dataframe(inscription_reprocessed, df_inscriptions)